# Use DwC-A_dotnet with F#

DwC-A_dotnet can be used with F# as well as C#.  The NuGet library installation and ArchiveReader/FileReader formatters work in the same way as they do for C#.

Here we'll use the Papilionidae dataset to demonstrate reading latitude and longitude information from the occurrence data file and plot it on a map of Texas with Plotly.

In [ ]:
#r "nuget:Plotly.NET,2.0.0-beta8"
#r "nuget:Plotly.NET.Interactive,2.0.0-beta8"
#r "nuget:FSharp.Data"
#r "nuget:DwC-A_dotnet,0.5.1"
#r "nuget:DwC-A_dotnet.Interactive,0.1.6-Pre"

In [ ]:
open DwC_A
open DwC_A.Terms
open DwC_A.Factories
open DwC_A.Config
open System.IO

let archiveFile = "./data/Papilionidae.zip"
let factory = new DefaultFactory(fun cfg -> 
    cfg.Add<ArchiveFolderConfiguration>( fun cfg -> 
        cfg.OutputPath <- "./Papilionidae"
        Directory.Delete(cfg.OutputPath, true) ))
let archive = new ArchiveReader(archiveFile, factory);

let occurrence = archive.CoreFile;
occurrence

In [ ]:
open System.Linq
open Plotly.NET

let lonlat = occurrence.DataRows
                    .Where(fun row -> row.[Terms.decimalLongitude] <> null && row.[Terms.decimalLatitude] <> null)
                    .Select(fun row -> ( $"{row.[Terms.genus]} {row.[Terms.specificEpithet]}", 
                                            row.[Terms.decimalLongitude] |> double, 
                                            row.[Terms.decimalLatitude] |> double) )
                    .GroupBy(fun row -> match row with (a, b, c) -> a)
                    .Select(fun group -> (group.Key, group.Select(fun row -> match row with (a, b, c) -> (b, c))))

let geo = lonlat.Select(fun row -> 
                            match row with (a, b) -> 
                                            Chart.ScatterGeo(b, 
                                                mode=StyleParam.Mode.Markers,
                                                Showlegend = true)
                                                |> Chart.withMarkerStyle(Size = 2) 
                                                |> Chart.withTraceName(a))
                                                |> Chart.Combine

let map = geo |> Chart.withMapStyle(
                FitBounds = StyleParam.GeoFitBounds.GeoJson,
                Scope = StyleParam.GeoScope.Usa,
                ShowLakes = true,
                ShowRivers = true,
                ShowLand = true,
                LandColor = "#f1f1f1"
            )
            |> Chart.withSize(height = 500.0, width = 800.0)
            |> Chart.withTitle(title = "Papilionidae of Texas")

map                 